In [1]:
from whispr import *
import pandas as pd

In [2]:
%%capture

'''
To add:


* update volumes on source plate at end (requires better keeping track of volume used)

* better commenting, use of functions / improving of readability

'''



In [3]:
"""
For user to change according to run
"""


file_name = "C:/Users/rycar/Documents/GitHub/WHISPR/source_plates/220105_exCRISPRa.xlsx"
#CSV file with source plate info. Columns are label, well, concentration, volume
source_plate_df = pd.read_excel(file_name, sheet_name = 'ECHO_source_plate', index_col = 0)

# CSV file with output plate layout: columns labeled 1-12 and rows labeled A-H for 96 well plate
output_plate = pd.read_excel(file_name, sheet_name = "plate_layout", index_col = 0, dtype = str)

#CSV file with each reaction as rows and the volume of each input (columns) added to reach final reaction volume
mixing_table_df = pd.read_excel(file_name, sheet_name = "mixing_table", index_col = 0, dtype = str).fillna(0)

source_plate_type = '384PP_AQ_BP' 

checkInputs(source_plate_df, mixing_table_df,source_plate_type)

#specify rxn_vol (default = 2.5) and total_vol (default = 10) if you'd like to change the volume of each individual replicate or the total reaction volume
vol_table_df = generateVolumeTable(mixing_table_df, source_plate_df)

output_df = writeProtocol(source_plate_type, vol_table_df, source_plate_df, output_plate,source_plate_df)
filename = '220102_exCRISPRa.csv'
output_df.to_csv('protocols/'+filename,index = False)

['C5', ' A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12']


c:\Users\rycar\Documents\GitHub\WHISPR\whispr.py:244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  source_plate_df['Well'][component] = ','.join(source_well[1:]).replace(' ','')


['A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'A11', 'A12']
Volumes used from each well for this protocol:
{'C5': 0.4, 'A2': 3.75, 'A3': 7.3, 'C14': 0.5, 'C15': 1.7, 'C17': 0.075, 'D1': 0.5, 'D8': 3.675, 'D12': 1.1, 'D23': 0.5, 'B14': 4.675, 'B15': 1.1, 'B16': 14.9, 'B17': 1.1, 'B20': 0.075, 'E1': 0.8, 'E2': 0.15, 'F3': 0.2}


In [15]:
mixing_table_df.columns
present = [m in source_plate_df.index for m in mixing_table_df.columns]

In [21]:
mixing_table_df.columns

Index(['pRC025', 'pRC027', 'pRC042', 'pRC043', 'pRC044', 'pRC045', 'pRC029',
       'pDA010.BL117-mCherry', 'pDA010.EL222', 'pDA010.188', 'pRC008.215',
       'pBT009.J1.119.Da9', ' pDA313', 'pDA307', 'pBT009.J1.119.DA6', 'ABA',
       'GA'],
      dtype='object')

In [26]:
    vol = []
    for component in list(source_plate_df['Volume']):
        if type(component) == str:
            c = component.split(',')
            for i in c:
                vol.append(float(i))
        else:
            vol.append(component)

In [28]:
vol_min = 20
if any([v < vol_min for v in vol]):
    print('yes')